In [1]:
#Imports
import scipy
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import shapiro,ttest_ind, levene, bartlett, f_oneway
from statsmodels.formula.api import ols

#### Testar se existe uma diferença significativa nas alturas médias de plantas tratadas com dois diferentes tipos de fertilizantes 

In [3]:
#Amostra de dados criada
#Altura das plantas (em cm) para cada tipo de fertilizante
alturas_fertilizantes_A = np.array([20, 21, 22, 24, 26, 28, 23, 25, 27, 29])
alturas_fertilizantes_B = np.array([19, 20, 21, 22, 24, 25, 26, 27, 28, 30])

In [4]:
#Realizando o teste t de duas amostras independentes
#A função retorna dois valores, a estatística do teste e o valor p
t_stat, p_valor = ttest_ind(alturas_fertilizantes_A, alturas_fertilizantes_B)

As hipóteses desse teste são formuladas da seguinte maneira:
* Hipótese Nula(H0) : Não há diferença significativa entre as médias das duas populações
* Hipótese alternativa(H1 ou Ha) : Existe uma diferença significativa entre as médias das duas populações

O teste t de duas amostras independentes pressupoe que as amostras são aleatórias, independentes entre si e que as populações de origem têm distribuições normais com variâncias iguais (homocedasticidade). Quando essas suposições não são atendidas, podem ser necessarios testes alternativos ou ajustes nos procedimentos.

O valor -p no teste t de duas amostras independentes é usado para determinar se a diferença observada entre as médias das duas amostras é estatisticamente significativa. A interpretação do valor-p depende do nível de significância escolhido para o teste, que é geralmente indicado por α (alfa). O nível de significância é a probabilidade de rejeitar a hipótese nula quando ela é verdadeira (erro do tipo I). Os valores de α mais comuns são 0,05 (5%) ou 0,01 (1%).

Aqui está como você pode interpretar o valor-p:

- Se o valor-p ≤ α: Há evidências estatísticas suficientes para rejeitar a hipótese nula. Isso significa que existe uma diferença significativa entre as médias das duas populações. A natureza dessa diferença (se uma média é maior ou menor que a outra) dependerá da direção da hipótese alternativa.

- Se o valor-p > α: Não há evidências estatísticas suficientes para rejeitar a hipótese nula. Isso significa que não há diferença significativa entre as médias das duas populações, com base nos dados da amostra e no nível de significância escolhido.




In [6]:
print(f"Estatistica t: {t_stat:.3f}")
print(f"Valor-p: {p_valor:.3f}")

Estatistica t: 0.200
Valor-p: 0.844


A estatística t no teste t de duas amostras independentes é uma medida da diferença entre as médias das duas amostras em relação à variabilidade combinada das duas amostras.

**Valor de t próximo de zero** : Indica que a diferença entre as médias das duas amostras é pequena em relaçãoo à variabilidade das amostras. Isso sugere que não há uma diferença significativa entre as duas populações.

**Valor de t positivo** : Indica que a média da primeira amostra é maior do que a a média da segunda amostra. Quanto maior o valor de t, maior é a diferença entre as médias em relação a variabilidade

**Valor de t negativo**: Indica que a média da primeira amostra é menor do que a média da segunda amostra. Novamente, quanto maior o valor absoluto de t, maior é a diferença entre as médias em relação a variabilidade

In [8]:
#Interpretando o resultado
nivel_significancia = 0.05

if p_valor <= nivel_significancia:
    print("Há evidências suficientes para rejeitar a hipótese nula. Existe uma diferença significativa entre as médias.")
else:
    print("Não há evidências suficientes para rejeitar a hipótese nula. Não existe uma diferença significativa entre as médias.")

Não há evidências suficientes para rejeitar a hipótese nula. Não existe uma diferença significativa entre as médias.


> Entretanto, não checamos uma das suposições do Teste t de Student, que as variações seguem uma distribuição normal. Podemos  fazer essa checagem usando o Teste de Shapiro Wilk

### Teste de Shapiro Wilk ( Checando a normalidade dos dados)

In [11]:
#Teste de Shapiro Wilk para verificar a normalidade
#Faço o teste para cada amostra de dados
#Ele vai retornar a estatística do teste e o valor p
stat_A, p_A = shapiro(alturas_fertilizantes_A)
stat_B, p_B = shapiro(alturas_fertilizantes_B)

O teste shapiro é aplicado separadamente aos dados de altura das plantas para cada tipo de fertilizante. Se o valor -p obtido for maior que o nível de significância (tipicamente 0.05), então não temos evidências suficientes para rejeitar a hipótese nula de normalidade, sugerindo que os dados podem ser considerados normalmente distribuídos. Se o valor -p for menor que o nível de significância, há evidências suficientes para rejeitar a hipótese de normalidade, indicando que os dados podem não seguir uma distribuição normal.

Lembrando que a decisão de usar o teste t de student depende fortemente da suposição de normalidade dos dados. Se essa suposição for violada, outros testes, como testes não paramétricos, podem ser mais adequados.

In [13]:
print(f"Resultado do teste de Shapiro-Wilk para o fertilizante A: Estatística ={stat_A:.3f}, Valor-p={p_A:.3f}")
print(f"Resultado do teste de Shapiro-Wilk para o fertilizante  B: Estatística={stat_B:.3f}, Valor-p={p_B:.3f}")

Resultado do teste de Shapiro-Wilk para o fertilizante A: Estatística =0.970, Valor-p=0.892
Resultado do teste de Shapiro-Wilk para o fertilizante  B: Estatística=0.968, Valor-p=0.872


In [14]:
#Interpretando os resultados

nivel_significancia = 0.05

if p_A > nivel_significancia:
    print("Não há evidências suficientes para rejeitar a hipótese de normalidade para o fertilizante A.")
else:
    print("Há evidências suficientes para rejeitar a hipótese de normalidade para o fertilizante A.")

if p_B > nivel_significancia:
    print("Não há evidências suficientes para rejitar a hipótese de normalidade para o fertilizante B.")
else:
    print("Há evidências suficientes para rejeitar a hipótese de normalidade para o fertilizante B.")

Não há evidências suficientes para rejeitar a hipótese de normalidade para o fertilizante A.
Não há evidências suficientes para rejitar a hipótese de normalidade para o fertilizante B.


## Teste F (ANOVA de um fator)

O Teste F, também conhecido como análise de variância (ANOVA), é um procedimento estatístico usado para comparar as médias de três ou mais grupos para determinar se pelo menos um dos grupos difere significativamente dos outros em termos de sua média. O Teste F avalia a hipótese nula de que todas as populações das quais as amostras foram retiradas têm a mesma média, contra a hipótese alternativa de que pelo menos uma média de população é diferente.

O valor F é calculado dividindo a variância entre os grupos pela variância dentro dos grupos. Se a variância entre os grupos é significativamente maior do que a variância dentro dos grupos, isso sugere que pelo menos um dos grupos difere dos outros em termos de sua média, levando à rejeição da hipótese nula.orietas.

### Testar se três diferentes dietas têm efeitos distintos sobre o peso de grupos de indivíduos. ###

As três dietas são o Grupo A, Grupo B e Grupo C. Você mediu o peso perdido por um mês em cada grupo e deseja saber se há diferenças significativas entre os efeitos das dietas.

In [25]:
#Peso perdido por indivíduos em cada grupo de dieta (em kg)
peso_perdido_A = np.array([2.1, 1.9, 2.2, 2.4, 2.3, 2.1, 2.2, 2.0])
peso_perdido_B = np.array([1.5, 1.7, 1.6, 1.8, 1.5, 1.7, 1.6])
peso_perdido_C = np.array([3.1, 3.2, 3.0, 3.3, 3.3, 3.1, 3.2, 3.1, 3.0])

In [27]:
# Realizando ANOVA de um fator
#Coloco as três amostras de dados e faz o teste
#Gera a estatística F e o valor P
f_stat, p_valor = f_oneway(peso_perdido_A, peso_perdido_B, peso_perdido_C)

In [29]:
print(f"Estatistica F: {f_stat:.3f}")
print(f"Valor-p: {p_valor:.3f}")

Estatistica F: 283.717
Valor-p: 0.000


In [31]:
#Interpretando o resultado

nivel_significancia = 0.05

if p_valor <= nivel_significancia:
    print("Há evidências suficientes para rejeitar a hipótese nula. Pelo menos uma das dietas tem um efeito significativamente diferente sobre o peso perdido.")
else:
    print("Não há evidências suficientes para rejeitar a hipótese nula. Não há diferenças significativas entre os efeitos das dietas no peso perdido.")

Há evidências suficientes para rejeitar a hipótese nula. Pelo menos uma das dietas tem um efeito significativamente diferente sobre o peso perdido.
